In [22]:
# !pip install gradio pandas
from typing import List, Dict
from dataclasses import dataclass
import json
import random
import gradio as gr
import pandas as pd

In [23]:
# Load datasets
patient_data = pd.read_csv('diet_recommendations_dataset.csv')
food_data = pd.read_csv('food.csv')

In [24]:
# Infer dietary tags based on Category and Description
def infer_tags(row):
    tags = []
    category = str(row['Category']).lower()
    description = str(row['Description']).lower()

In [25]:
# Infer dietary tags based on Category and Description
def infer_tags(row):
    tags = []
    category = str(row['Category']).lower()
    description = str(row['Description']).lower()

    # Dietary categories
    if 'cheese' in category or 'milk' in category or 'cream' in category or 'butter' in category:
        tags.append('dairy')
    if 'pork' in category or 'beef' in category or 'chicken' in category or 'turkey' in category:
        tags.append('meat')
    if 'vegetarian' not in tags and 'meat' not in tags and 'dairy' not in tags:
        tags.append('vegan')  # Assume vegan if no dairy or meat
    if 'meat' not in tags:
        tags.append('vegetarian')  # Non-meat is vegetarian

    # Allergies
    if 'peanut' in description:
        tags.append('peanuts')
    if 'soy' in description:
        tags.append('soy')

    # Diet recommendations
    if row['Data.Major Minerals.Sodium'] < 100:  # Low sodium threshold
        tags.append('low_sodium')
    if row['Data.Carbohydrate'] < 10:  # Low carb threshold
        tags.append('low_carb')
    if row['Data.Sugar Total'] < 5:  # Low sugar for diabetes
        tags.append('low_sugar')

    return tags

In [26]:
# Convert food dataset to nutritional_db format
nutritional_db = {}
for _, row in food_data.iterrows():
    food_name = row['Description']
    nutritional_db[food_name] = {
        'calories': float(row['Data.Kilocalories']) if pd.notna(row['Data.Kilocalories']) else 0.0,
        'protein': float(row['Data.Protein']) if pd.notna(row['Data.Protein']) else 0.0,
        'carbs': float(row['Data.Carbohydrate']) if pd.notna(row['Data.Carbohydrate']) else 0.0,
        'fat': float(row['Data.Fat.Total Lipid']) if pd.notna(row['Data.Fat.Total Lipid']) else 0.0,
        'tags': infer_tags(row),
        'portion_size': float(row['Data.Household Weights.1st Household Weight']) if pd.notna(row['Data.Household Weights.1st Household Weight']) else 100.0,
        'portion_description': str(row['Data.Household Weights.1st Household Weight Description']) if pd.notna(row['Data.Household Weights.1st Household Weight Description']) else '100g'
    }

In [27]:
@dataclass
class UserProfile:
    name: str
    age: int
    weight: float
    height: float
    gender: str
    activity_level: str
    goal: str
    restrictions: List[str]
    conditions: List[str]

def calculate_bmr(user: UserProfile) -> float:
    if user.gender.lower() == 'male':
        bmr = 10 * user.weight + 6.25 * user.height - 5 * user.age + 5
    else:
        bmr = 10 * user.weight + 6.25 * user.height - 5 * user.age - 161
    return bmr

In [28]:
def calculate_daily_calories(user: UserProfile) -> float:
    activity_multipliers = {
        "sedentary": 1.2,
        "light": 1.375,
        "moderate": 1.55,
        "active": 1.725
    }
    multiplier = activity_multipliers.get(user.activity_level.lower(), 1.2)
    base_calories = calculate_bmr(user) * multiplier
    if user.goal.lower() == "weight loss":
        return base_calories - 500
    elif user.goal.lower() == "muscle gain":
        return base_calories + 500
    else:
        return base_calories

In [38]:
def filter_foods(user: UserProfile) -> List[str]:
    allowed_foods = []
    for food, details in nutritional_db.items():
        # Check dietary restrictions
        if user.restrictions:
            if not all(r.lower() in details['tags'] for r in user.restrictions if r.lower() in ['vegan', 'vegetarian', 'low_sodium', 'low_carb']):
                continue
            if any(r.lower() in details['tags'] for r in user.restrictions if r.lower() in ['peanuts', 'soy']):
                continue
        # Check health conditions
        if "diabetes" in [c.lower() for c in user.conditions] and details["carbs"] > 25:
            continue
        if "cardiovascular" in [c.lower() for c in user.conditions] and details["fat"] > 15:
            continue
        allowed_foods.append(food)
    return allowed_foods

In [39]:
def generate_meal_plan(user: UserProfile) -> Dict[str, Dict[str, float]]:
    daily_calories = calculate_daily_calories(user)
    allowed_foods = filter_foods(user)
    meal_plan = {}
    total = {"calories": 0.0, "protein": 0.0, "carbs": 0.0, "fat": 0.0}

    attempts = 0
    max_attempts = 100

    while total['calories'] < daily_calories - 100 and allowed_foods and attempts < max_attempts:
        food = random.choice(allowed_foods)
        attempts += 1
        if food in meal_plan:
            continue
        # Adjust nutrients based on portion size (per 100g to portion size)
        portion_factor = nutritional_db[food]['portion_size'] / 100.0
        meal_plan[food] = {
            'calories': nutritional_db[food]['calories'] * portion_factor,
            'protein': nutritional_db[food]['protein'] * portion_factor,
            'carbs': nutritional_db[food]['carbs'] * portion_factor,
            'fat': nutritional_db[food]['fat'] * portion_factor,
            'portion': nutritional_db[food]['tags'],
            'portion_size': nutritional_db[food]['portion_size'],
            'portion_description': nutritional_db[food]['portion_description']
        }
        for key in total:
            total[key] += meal_plan[food].get(key, 0.0)

    return {"meals": meal_plan, "totals": total, "goal": daily_calories}

In [44]:
def create_diet_plan(patient_id: str, user_data: dict) -> Dict:
    # Fetch patient data
    patient = patient_data[patient_data['Patient_ID'] == patient_id]
    if patient.empty:
        return {"error": ["Invalid Patient ID"]}


In [45]:
# Add proper error handling and data validation
def create_diet_plan(patient_id: str, user_data: dict) -> Dict:
    try:
        # Fetch patient data
        patient = patient_data[patient_data['Patient_ID'] == patient_id]
        if patient.empty:
            return {"error": ["Invalid Patient ID"]}
        
        # Debug: Check what data we're working with
        # print(f"Patient found: {patient.iloc[0]}")
        
        # Map patient data to UserProfile - FIXED: Handle NaN values properly
        restrictions = []
        
        # Handle Dietary_Restrictions
        dietary_restrictions = patient['Dietary_Restrictions'].iloc[0]
        if pd.notna(dietary_restrictions) and str(dietary_restrictions) != "None" and str(dietary_restrictions) != "nan":
            restrictions.append(str(dietary_restrictions).lower())
        
        # Handle Allergies
        allergies = patient['Allergies'].iloc[0]
        if pd.notna(allergies) and str(allergies) != "None" and str(allergies) != "nan":
            restrictions.append(str(allergies).lower())
        
        # Handle Diet_Recommendation
        diet_recommendation = patient['Diet_Recommendation'].iloc[0]
        if pd.notna(diet_recommendation) and str(diet_recommendation).lower() in ["low_sugar", "low_carb"]:
            restrictions.append(str(diet_recommendation).lower())

        conditions = []
        # Handle Disease_Type
        disease_type = patient['Disease_Type'].iloc[0]
        if pd.notna(disease_type) and str(disease_type).lower() in ["diabetes", "cardiovascular"]:
            conditions.append(str(disease_type).lower())

        # Merge with user-provided restrictions and conditions
        user_restrictions = user_data.get('restrictions', [])
        user_conditions = user_data.get('conditions', [])
        
        all_restrictions = list(set(restrictions + user_restrictions))
        all_conditions = list(set(conditions + user_conditions))

        user_data.update({
            "restrictions": all_restrictions,
            "conditions": all_conditions,
        })

        user = UserProfile(**user_data)
        return generate_meal_plan(user)
        
    except Exception as e:
        return {"error": [f"Error processing patient data: {str(e)}"]}

In [46]:
def gradio_interface(patient_id, name, age, weight, height, gender, activity_level, goal, restrictions, conditions):
    try:
        user_data = {
            "name": name,
            "age": int(age),
            "weight": float(weight),
            "height": float(height),
            "gender": gender,
            "activity_level": activity_level,
            "goal": goal,
            "restrictions": [r.strip().lower() for r in restrictions.split(',') if r.strip()],
            "conditions": [c.strip().lower() for c in conditions.split(',') if c.strip()]
        }
        
        result = create_diet_plan(patient_id, user_data)
        
        if "error" in result:
            return f"❌ Error\n\n{result['error'][0]}"
        
        # Create markdown output
        markdown_output = f"""
🍽️ PERSONALIZED DIET PLAN

👤 Patient Information
- Name: {user_data['name']}
- Daily Calorie Goal: {result['goal']:.0f} calories

📋 Recommended Meals
"""
        
        meal_number = 1
        for food, details in result['meals'].items():
            markdown_output += f"""
{meal_number}. {food}
- Portion: {details['portion_size']:.1f}g ({details['portion_description']})
- Calories: {details['calories']:.0f} kcal
- Protein: {details['protein']:.1f}g
- Carbs: {details['carbs']:.1f}g
- Fat: {details['fat']:.1f}g
"""
            meal_number += 1
        
        markdown_output += f"""
📊 Daily Nutrition Totals
- Total Calories: {result['totals']['calories']:.0f} kcal
- Total Protein: {result['totals']['protein']:.1f}g
- Total Carbs: {result['totals']['carbs']:.1f}g
- Total Fat: {result['totals']['fat']:.1f}g

💡 Nutritional Insights
"""
        
        # Add nutritional insights
        protein_percent = (result['totals']['protein'] * 4 / result['totals']['calories']) * 100
        carbs_percent = (result['totals']['carbs'] * 4 / result['totals']['calories']) * 100
        fat_percent = (result['totals']['fat'] * 9 / result['totals']['calories']) * 100
        
        markdown_output += f"""
- Macronutrient Distribution:
  - Protein: {protein_percent:.1f}%
  - Carbohydrates: {carbs_percent:.1f}%
  - Fat: {fat_percent:.1f}%

- Recommendations:
  - {'✅ Excellent protein intake' if protein_percent > 20 else '⚠️ Consider more protein'}
  - {'✅ Balanced carbs' if 40 <= carbs_percent <= 60 else '⚠️ Adjust carb intake'}
  - {'✅ Healthy fat levels' if 20 <= fat_percent <= 35 else '⚠️ Monitor fat intake'}
"""
        
        return markdown_output
        
    except Exception as e:
        return f"❌ Error\n\n{str(e)}"



In [47]:
def filter_foods(user: UserProfile) -> List[str]:
    allowed_foods = []
    for food, details in nutritional_db.items():
        # Skip foods with missing nutritional data
        if any(pd.isna(val) for val in [details['calories'], details['protein'], details['carbs'], details['fat']]):
            continue
            
        # Check dietary restrictions
        skip_food = False
        
        # Handle vegan/vegetarian restrictions
        if 'vegan' in user.restrictions and ('dairy' in details['tags'] or 'meat' in details['tags']):
            skip_food = True
        elif 'vegetarian' in user.restrictions and 'meat' in details['tags']:
            skip_food = True
            
        # Handle allergy restrictions
        for restriction in user.restrictions:
            if restriction in ['peanuts', 'soy'] and restriction in details['tags']:
                skip_food = True
                break
                
        # Handle low sodium/carb/sugar restrictions
        if 'low_sodium' in user.restrictions and not any('low_sodium' in tag for tag in details['tags']):
            skip_food = True
        if 'low_carb' in user.restrictions and not any('low_carb' in tag for tag in details['tags']):
            skip_food = True
        if 'low_sugar' in user.restrictions and not any('low_sugar' in tag for tag in details['tags']):
            skip_food = True
            
        # Check health conditions
        if "diabetes" in user.conditions and details["carbs"] > 25:
            skip_food = True
        if "cardiovascular" in user.conditions and details["fat"] > 15:
            skip_food = True
            
        if not skip_food:
            allowed_foods.append(food)
            
    return allowed_foods

In [ ]:
# Create a more robust Gradio interface
demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Patient ID", placeholder="e.g., P0001", value="P0001"),
        gr.Textbox(label="Name", value="John Doe"),
        gr.Number(label="Age", value=30),
        gr.Number(label="Weight (kg)", value=70),
        gr.Number(label="Height (cm)", value=175),
        gr.Radio(["Male", "Female"], label="Gender", value="Male"),
        gr.Dropdown(["Sedentary", "Light", "Moderate", "Active"], label="Activity Level", value="Moderate"),
        gr.Dropdown(["Weight Loss", "Maintain", "Muscle Gain"], label="Fitness Goal", value="Maintain"),
        gr.Textbox(label="Additional Dietary Restrictions (comma-separated)", 
                  placeholder="e.g., vegan, low_sodium", value=""),
        gr.Textbox(label="Additional Health Conditions (comma-separated)", 
                  placeholder="e.g., diabetes, cardiovascular", value="")
    ],
    outputs=gr.Textbox(label="Personalized Diet Plan"),
    title="AI Diet Assistant",
    description="Get a personalized diet plan based on your health profile and goals."
)

if __name__ == "__main__":
    demo.launch(inline=True, share=True)

* Running on local URL:  http://127.0.0.1:7870
* Running on public URL: https://9ceadd87a3e8d1c50f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Using existing dataset file at: .gradio\flagged\dataset4.csv
